In [27]:
import cv2
import psycopg2
from datetime import datetime
from ultralytics import YOLO
from psycopg2.pool import SimpleConnectionPool
from threading import Lock
import os
import queue
import threading
from tqdm import tqdm

CLASS_NAMES = ["person", "canopy", "pallet"]
DATABASE_CONFIG = {
    "dbname": "market_db",
    "user": "test_user",
    "password": "12345",
    "host": "localhost",
    "port": "5432"
}

# Создание пула соединений
connection_pool = SimpleConnectionPool(
    minconn=1,
    maxconn=10,
    **DATABASE_CONFIG
)

# Очередь для асинхронной записи
event_queue = queue.Queue()
lock = Lock()

# Словарь для отслеживания состояния объектов
tracked_objects = {
    "person": {},  # {track_id: (last_seen_frame, is_reported)}
    "pallet": {}
}

# Словарь для преобразования class_name в event_type
EVENT_TYPE_MAP = {
    "person": "person_detected",
    "pallet": "pallet_detected"
}


def calculate_iou(box1, box2):
    # Координаты пересечения
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    # Площадь пересечения
    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    inter_area = inter_width * inter_height

    # Площади каждого прямоугольника
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area if union_area > 0 else 0
    return iou


def init_db():
    conn = connection_pool.getconn()
    try:
        with conn.cursor() as cursor:
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS events (
                    id SERIAL PRIMARY KEY,
                    event_time TIMESTAMP NOT NULL,
                    event_type VARCHAR(50) NOT NULL CHECK (
                        event_type IN (
                            'person_detected',
                            'pallet_detected',
                            'pallet_covered'
                        )
                    ),
                    photo_path VARCHAR(255) NOT NULL
                );
            """)
            conn.commit()
    finally:
        connection_pool.putconn(conn)


# Поток для записи событий в БД
def save_events_worker():
    while True:
        events = event_queue.get()
        if events is None:
            break

        conn = connection_pool.getconn()
        try:
            with conn.cursor() as cursor:
                for event in events:
                    cursor.execute(
                        "INSERT INTO events (event_time, event_type, photo_path) VALUES (NOW(), %s, %s)",
                        event
                    )
                conn.commit()
        except Exception as e:
            print(f"Ошибка записи: {e}")
            conn.rollback()
        finally:
            connection_pool.putconn(conn)
            event_queue.task_done()


def calculate_coverage(inner_box, outer_box):
    """
    Вычисление покрытия внутреннего прямоугольника (паллет) внешним (купол)
    Возвращает долю площади inner_box, покрытую outer_box
    """
    # Координаты пересечения
    x_left = max(inner_box[0], outer_box[0])
    y_top = max(inner_box[1], outer_box[1])
    x_right = min(inner_box[2], outer_box[2])
    y_bottom = min(inner_box[3], outer_box[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Площадь пересечения
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Площадь паллета
    inner_area = (inner_box[2] - inner_box[0]) * (inner_box[3] - inner_box[1])

    if inner_area == 0:
        return 0.0

    return intersection_area / inner_area


# Обработка видео и детекция событий
def process_video(video_path):
    model = YOLO("runs/final_train/weights/best.pt")
    cap = cv2.VideoCapture(video_path)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count = 0

    db_thread = threading.Thread(target=save_events_worker, daemon=True)
    db_thread.start()

    with tqdm(total=total_frames, desc="Processing frames", unit="frame") as pbar:
        while cap.isOpened():
            success, frame = cap.read()
            frame_count += 1
            if not success:
                break

            pbar.update(1)

            results = model.track(frame, persist=True, verbose=False, conf=0.5)
            boxes = results[0].boxes.xyxy.cpu().numpy()
            classes = results[0].boxes.cls.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int) if results[0].boxes.id is not None else []

            current_objects = {"person": {}, "canopy": {}, "pallet": {}}
            events = []

            # Собираем информацию о текущих объектах
            for box, cls, obj_id in zip(boxes, classes, ids):
                class_name = CLASS_NAMES[cls]
                current_objects[class_name][obj_id] = box

            # Обработка людей и паллетов
            for class_name in ["person", "pallet"]:
                for obj_id in current_objects[class_name]:
                    if obj_id not in tracked_objects[class_name]:
                        tracked_objects[class_name][obj_id] = {
                            "last_seen": frame_count,
                            "reported": False,
                            "covered": False  # Только для паллетов
                        }
                    else:
                        tracked_objects[class_name][obj_id]["last_seen"] = frame_count

                    # Обработка новых объектов
                    if not tracked_objects[class_name][obj_id]["reported"]:
                        event_type = EVENT_TYPE_MAP.get(class_name)
                        events.append((event_type, frame))
                        tracked_objects[class_name][obj_id]["reported"] = True

            # Проверка покрытия паллетов куполами
            for pallet_id in current_objects["pallet"]:
                pallet_box = current_objects["pallet"][pallet_id]
                max_coverage = 0.0

                # Ищем максимальное покрытие любым куполом
                for canopy_box in current_objects["canopy"].values():
                    coverage = calculate_coverage(pallet_box, canopy_box)
                    max_coverage = max(max_coverage, coverage)

                # Проверяем состояние покрытия
                is_covered = max_coverage > 0.7  # Порог покрытия 70%
                prev_state = tracked_objects["pallet"][pallet_id].get("covered", False)

                if is_covered and not prev_state:
                    events.append(("pallet_covered", frame))
                    tracked_objects["pallet"][pallet_id]["covered"] = True
                elif not is_covered and prev_state:
                    tracked_objects["pallet"][pallet_id]["covered"] = False

            # Очистка устаревших объектов
            for class_name in ["person", "pallet"]:
                to_delete = []
                for obj_id in tracked_objects[class_name]:
                    if frame_count - tracked_objects[class_name][obj_id]["last_seen"] > 10:
                        to_delete.append(obj_id)

                for obj_id in to_delete:
                    del tracked_objects[class_name][obj_id]

            if not os.path.exists("events"):
                os.makedirs("events")

            # Сохранение событий
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
            with lock:
                for event_type, frame in events:
                    annotated_frame = results[0].plot()
                    photo_path = f"events/{timestamp}_{event_type}_{frame_count}.jpg"
                    cv2.imwrite(photo_path, annotated_frame)
                    event_queue.put([(event_type, photo_path)])

    cap.release()
    event_queue.put(None)
    db_thread.join()


if __name__ == "__main__":
    init_db()
    process_video("data/train.mp4")
    connection_pool.closeall()

Processing frames: 100%|██████████| 11880/11880 [07:54<00:00, 25.05frame/s]
